In [1]:
from utility.ProteinParser import ProteinData
from utility.ProteinModel import ProteinModel
from utility.ProteinStructurePrediction import ProteinStructurePrediction
# from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

Using TensorFlow backend.
/opt/conda/envs/qfold/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


# Step 1: Prepare Data

In this part, we load the folder with the raw rna data for experimentation. To evaluate the quantum solution and the actual solution include both fasta and ct files. To only generate a quantum solution, upload a fasta file.


In [2]:
# input: aminoacids
# output: energy files

In [3]:
data_path='protein-folding-data'

# Step 2: Build Model

In this part, we will show how to build model for qfold

In [4]:
# initial the ProteinFold object
init_param = {}
# method: qfold-cc stands for the classical metropolis method in QFold
# method: qfold-qc stands for the quantum metropolis method in QFold
method = ['qfold-cc', 'qfold-qc']

for mt in method:
    if mt == 'qfold-cc':
        init_param[mt] = {}
        init_param[mt]['params'] = ["initialization"]
    elif mt == 'qfold-qc':
        init_param[mt] = {}
        init_param[mt]['params'] = ["initialization"]
    
protein_model = ProteinModel(data_path, method, **init_param)

INFO:root:Initial parameters for protein glycylglycine_3_GG using qfold-cc
INFO:root:Initial parameters for protein glycylglycine_3_GG using qfold-qc
INFO:root:Initial parameters for protein glycylglycine_4_GG using qfold-cc
INFO:root:Initial parameters for protein glycylglycine_4_GG using qfold-qc


In [5]:
# set the parameters for model
model_param = {}

method = 'qfold-cc'
model_param[method] = {}

# parameters
model_param[method]['initialization'] = ["minifold", "random"]

method = 'qfold-qc'
model_param[method] = {}

# parameters
model_param[method]['initialization'] = ["minifold", "random"]

protein_model.build_models(**model_param)


deltas_dict length for glycylglycine_3_GG: 256
deltas_dict length for glycylglycine_3_GG: 256
deltas_dict length for glycylglycine_4_GG: 1024
deltas_dict length for glycylglycine_4_GG: 1024
deltas_dict length for glycylglycine_3_GG: 256
deltas_dict length for glycylglycine_3_GG: 256
deltas_dict length for glycylglycine_4_GG: 1024
deltas_dict length for glycylglycine_4_GG: 1024


0

In [6]:
# save the model
model_path = protein_model.save("latest")

print(f"You have built the protein folding models and saved them as protein_folding_latest.pickle")

INFO:root:finish save protein_folding_latest.pickle


You have built the protein folding models and saved them as protein_folding_latest.pickle


# Step 3: Predict Protein Structure

In this part, we will show how to run models for predicting protein structure

In [7]:
protein_models = ProteinModel.load(model_path)

In [8]:
model_info = protein_models.describe_models()

INFO:root:debug describe
INFO:root:model name: glycylglycine_3_GG, method: qfold-cc
INFO:root:param: initialization, value {'minifold', 'random'}
INFO:root:model name: glycylglycine_3_GG, method: qfold-qc
INFO:root:param: initialization, value {'minifold', 'random'}
INFO:root:model name: glycylglycine_4_GG, method: qfold-cc
INFO:root:param: initialization, value {'minifold', 'random'}
INFO:root:model name: glycylglycine_4_GG, method: qfold-qc
INFO:root:param: initialization, value {'minifold', 'random'}


In [9]:
# get the model you want to optimize
protein_name = 'glycylglycine_3_GG'
initialization = 'random'
method = 'qfold-cc'

model_name = "{}+{}".format(protein_name, initialization)

protein_model = protein_models.get_model(protein_name, method, model_name)

In [10]:
# protein_model["model"].deltas_dict

In [11]:
data_path = 'data'
# psp_param stands for the parameters for predicting protein structure
psp_param = {}
psp_param["data_path"] = data_path
psp_param["mode"] = 'local-simulator'
psp_param["model_name"] = model_name
psp_param["model_path"] = model_path

psp = ProteinStructurePrediction(protein_model, method, **psp_param)

psp.run()

INFO:root:initial protein structure prediction using qfold-cc in QFold
INFO:root:CLASSICAL METROPOLIS: Time for 2 steps: 0.30298733711242676 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 3 steps: 0.42849063873291016 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 4 steps: 0.5459754467010498 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 5 steps: 0.661046028137207 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 6 steps: 0.7801580429077148 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 7 steps: 0.9172167778015137 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 8 steps: 1.0284547805786133 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 9 steps: 1.1629078388214111 seconds
INFO:root:finish save tts_results_glycylglycine_3_GG+random_1000_qfold-cc.json


In [12]:
initialization = 'random'
method = 'qfold-qc'

model_name = "{}+{}".format(protein_name, initialization)

protein_model = protein_models.get_model(protein_name, method, model_name)

In [13]:
psp = ProteinStructurePrediction(protein_model, method, **psp_param)

psp.run()

INFO:root:initial protein structure prediction using qfold-qc in QFold
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.18668 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05698 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.07486 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: UnitarySynthesis - 0.01144 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: UnrollCustomDefinitions - 27134.67717 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('x', 1), ('mcu1', 9), ('cx', 2), ('cu3', 2), ('snapshot', 16), ('ccx', 3), ('mcx', 5), ('mcx', 4), ('h', 1)} to target basis {'ryy', 'save_density_matrix', 'mcrz', 'ccx', 'save_probs', 'save_amplitudes', 'u2', 'rz', 'tdg', 'z', 'mcp', 'unitary', 'mcry', 'rxx', 'measure', 'rx', 'csx', 'delay', 'snapshot', 'r', 'quantum_channel', 'roerror', 'sdg', 'mcr', 's', 'barrier', 'mcu3', 'diagonal', 'sx', 'qerror_loc', 'ry', 'save_amplitudes_sq', 'h', 'mcswap', 'kraus', 'm

# Step 4: Post Process

In this part, we will show how to visualize the results